In [1]:
import numpy as np
import sympy as sp
import itertools as it
import string
import pickle
from tqdm import tqdm
import time
from scipy.io import savemat

In [2]:
#cyclic permutation to check equivalence under the parrtial trace
def cyclic_permutations(lst):
    yield lst
    for k in range(3, len(lst), 3):
        p = lst[k:] + lst[:k]
        if p == lst:
            break
        yield p

In [3]:
#cyclic permutation to check equivalence under the parrtial trace
def cyclic_permutations_clock(lst):
    yield lst
    for k in range(3, len(lst), 3):
        p = lst[-k:] + lst[:-k]
        if p == lst:
            break
        yield p

In [26]:
parties = ['A']
inputs = ['0','1']#,'2','3','4','5','6','7','8','9','B','C','D','E','F','G'
outputs = ['0','1']#,2','3','4','5','6','7','8','9','B','C','D','E','F'
operators = [''.join(k) for k in it.product(parties,outputs,inputs, repeat = 1)]
A_operators = [''.join(k) for k in it.product(parties,outputs,inputs, repeat = 1)]
#A_operators.append('100')

In [5]:
operators.append('R00')
#operators.append('100')


In [6]:
level = 2
monomial_set = []
lev=[]
for i in range(1,level+1):   
    
    lev = [''.join(k) for k in it.product(operators, repeat = i)]
    #lev = [''.join(k) for k in it.product(A_operators, repeat = i)]
    if(i==3):
        tmp= [''.join(k) for k in it.product(A_operators, repeat = 1)]
        for count1 in range(len(tmp)):
            lev.append('R00'+tmp[count1])
            lev.append('A00'+tmp[count1])
            lev.append('A10'+tmp[count1])
            lev.append(tmp[count1]+'A20')
            lev.append(tmp[count1]+'A30')
            lev.append('A01'+tmp[count1])
            lev.append('A02'+tmp[count1])
            lev.append('A03'+tmp[count1])
            lev.append('A04'+tmp[count1])
            #lev.append('A05'+tmp[count1])
            #lev.append(tmp[count1]+'A01')
            #lev.append(tmp[count1]+'A11')
            #for count2 in range(len(tmp)):
                #if(count1<count2):
                #    lev.append(tmp[count1]+tmp[count2]+tmp[count1]+tmp[count2])
                #lev.append(tmp[count1]+tmp[count2])
                #if(count1<count2):
                #    lev.append(tmp[count1]+tmp[count2])
                    #lev.append('R00'+tmp[count1]+tmp[count2])
                    #lev.append('A00'+tmp[count1]+tmp[count2])
                    #lev.append('A10'+tmp[count1]+tmp[count2])
                #for count3 in range(len(tmp)):
                    #if(count1<count2<count3):
                        #lev.append('R00'+tmp[count1])
                        #lev.append('A00'+tmp[count1])
                        #lev.append('A10'+tmp[count1])
                        #lev.append('A20'+tmp[count1])
                        #lev.append('A30'+tmp[count1])
                        #lev.append(tmp[count1]+tmp[count2]+tmp[count3])
                        



    #jointly measurable measurements commute
    #for k in range(len(lev)):
    #    for l1 in parties: 
    #        for i1 in inputs:
    #            for i2 in inputs:
    #                for o1 in outputs:
    #                    for o2 in outputs:
    #                        if i1>i2 and o1>non_joint:
    #                            lev[k] = lev[k].replace(l1+o1+i1+l1+o2+i2, l1+o2+i2+l1+o1+i1)

    ### orthogonality                                   
    for k in range(len(lev)):
        for l1 in parties: 
            for l2 in parties:
                for i1 in inputs:
                    for o1 in outputs:
                        for o2 in outputs:
                            if (o1!=o2 and l1+o1+i1+l2+o2+i1 in lev[k]):
                                lev[k]= ('')


            
                                                                                
    #if i == 1:
    #    operators = lev.copy()
    #every element projects into itself
    for p in range(len(lev)):      
        for m in range(1,int(i/2)+2):
            #s = [''.join(k) for k in it.product(operators, repeat = m)]
            while any(el+el in lev[p] for el in operators):
                for el in operators:
                    lev[p] = lev[p].replace(el+el, el)
    
    
    monomial_set += lev

monomial_set.append('')
monomial_set = sorted(sorted(list(set(monomial_set))), key=len)
#remove unnecessary elements
#monomial_set.remove('')
print(len(monomial_set))

22


In [7]:
n=3
rev_mon_set = []
for l in range(len(monomial_set)):
    lista = [monomial_set[l][i:i+n] for i in range(0, len(monomial_set[l]), n)]
    lista.reverse()
    rev_mon_set.append(''.join(lista))

In [8]:
gamma_mat = []
for i in range(len(monomial_set)):
    gamma_row = []
    for j in range(i, len(rev_mon_set)):
        gamma_row.append(rev_mon_set[i]+monomial_set[j])
    gamma_mat.append(gamma_row)

In [9]:
Rem_el=[]
for i in tqdm(range(len(gamma_mat))):
    #time.sleep(0.01)
    for j in range(len(gamma_mat[i])):

        #jointly measurable measurements commute
        #for k in range(int(len(gamma_mat[i][j])/2)+1):
        #    for l1 in parties: 
        #        for i1 in inputs:
        #            for i2 in inputs:
        #                for o1 in outputs:
        #                    for o2 in outputs:
        #                        if i1>i2 and o1>non_joint:
        #                            gamma_mat[i][j] = gamma_mat[i][j].replace(l1+o1+i1+l1+o2+i2, l1+o2+i2+l1+o1+i1)
        
        
        #setting orthogonal elements to 'O00'
        for l1 in parties: 
            for i1 in inputs:
                for o1 in outputs:
                    for o2 in outputs:
                        if (o1!=o2 and l1+o1+i1+l1+o2+i1 in gamma_mat[i][j]):
                            Rem_el.append(gamma_mat[i][j])
                            gamma_mat[i][j] = ('O00')

        #the partial trace is cyclical over the first party
        for l1 in parties: 
            for i1 in inputs:
                for o1 in outputs:
                    for o2 in outputs:
                        if (o1!=o2 and l1+o1+i1+l1+o2+i1 in gamma_mat[i][j][0:3]+gamma_mat[i][j][-3::]):
                            Rem_el.append(gamma_mat[i][j])
                            gamma_mat[i][j] = ('O00')

        #every element project into itself
        for m in range(1,int(len(gamma_mat[i][j])/2)+1):
            #s = [''.join(k) for k in it.product(operators, repeat = m)]
            while any(el+el in gamma_mat[i][j] for el in operators):
                for el in operators:
                    #print(gamma_mat[i][j], el+el, i,j)
                    gamma_mat[i][j] = gamma_mat[i][j].replace(el+el, el)
                    #print(gamma_mat[i][j])
                    #print('***')
        
        #the partial trace is cyclical over the first party
        for m in range(1,int(len(gamma_mat[i][j])/2)+1):
            while len(gamma_mat[i][j])>7 and any(el+el in gamma_mat[i][j][0:3]+gamma_mat[i][j][-3::] for el in A_operators):
                gamma_mat[i][j] = gamma_mat[i][j][:-3]

                

        
        #the partial trace is cyclical over the first party
        if ('R00' not in gamma_mat[i][j]):
            for tmp in cyclic_permutations(gamma_mat[i][j]):
                if (tmp<gamma_mat[i][j]):
                    gamma_mat[i][j]=tmp
        else:
            #tmp1='Z'*len(gamma_mat[i][j])
            #tmp2='Z'*len(gamma_mat[i][j])
            flag=0
            if(gamma_mat[i][j][0:3] != 'R00'):
                for tmp in cyclic_permutations(gamma_mat[i][j]):
                    if (tmp[0:3] == 'R00'):
                        tmp1=tmp
                        tmp=gamma_mat[i][j]
                        flag=1
                        break
            if(flag):
                gamma_mat[i][j]=tmp1

                

diag_el=[]
for i in range(len(gamma_mat)):
    diag_el.append(gamma_mat[i][0])

100%|████████████████████████████████████████████████████████████████████████████████| 22/22 [00:00<00:00, 1391.94it/s]


In [10]:
monomial_index = {}
for i in range(len(gamma_mat)):
    for j in range(len(gamma_mat[i])):
        monomial_index[gamma_mat[i][j]] = []

for i in range(len(gamma_mat)):
    for j in range(len(gamma_mat[i])):
        monomial_index[gamma_mat[i][j]].append([i,i+j])

In [11]:
diz_constraints = { list(monomial_index.keys())[i] : i  for i in range(len(list(monomial_index.keys())))}

In [12]:
index_keys = list(monomial_index.keys())

In [13]:
coefs = [item for item in index_keys if 'R00' not in item]
if('O00' in coefs):
    coefs.remove('O00')
coefs.remove('')
matrices = [item for item in index_keys if 'R00' in item]
hermitian_M=[]
complex_M=[]
real_C=[]
complex_C=[]

for i in matrices:
    lista = [i[j:j+n] for j in range(0, len(i), n)]
    lista.reverse()
    lista=''.join(lista)
    flag=0
    if(lista[0:3] != 'R00'):
        for tmp in cyclic_permutations(lista):
            if (tmp[0:3] == 'R00'):
                tmp1=tmp
                tmp=lista
                flag=1
                break
    if(flag):
        lista=tmp1
    if(lista == i):
        hermitian_M.append(i)


hermitian_M = sorted(sorted(set(hermitian_M)), key=len);
complex_M = [i for i in matrices if i not in hermitian_M]


for i in coefs:
    lista = [i[j:j+n] for j in range(0, len(i), n)]
    lista.reverse()
    lista = ''.join(lista)
    for tmp in cyclic_permutations(lista):
        if (tmp<lista):
            lista=tmp
    if(lista==i):
        real_C.append(i)


real_C=sorted(sorted(set(real_C)), key=len);
complex_C = [i for i in coefs if i not in real_C]

In [14]:
#REDUCTION RULE DEFINED IN SUPPLEMENTARY MATHERIAL FOR DIMENSION 2, REMOVE IF NEEDED
#count=0
#for i in complex_C:
#    if(len(i)==12):
#        if(len({i[2], i[5], i[8], i[-1]})<4):
#            count=count+1
#            real_C.append(i)
#            print(i, i[2], i[5], i[8], i[-1])
#
#real_C=sorted(sorted(set(real_C)), key=len);
#complex_C = [i for i in coefs if i not in real_C]
#count

In [15]:
#constrained_el=[]
#constrained_el.append(0)
#for i in [1, *[i for i in range(2, (level+2)*2) if i % 2 == 0]]:
#    constrained_el.append(len([item for item in real_C if len(item)==3*i]))
#for i in range(len(constrained_el)-1, 1, -1):
#    constrained_el[i]=constrained_el[i]+sum(constrained_el[j] for j in range(i))

In [16]:
C = np.zeros([len(real_C),len(monomial_set),len(monomial_set)], dtype=int)
for i in range(len(real_C)):
    for j in range(len(monomial_index[real_C[i]])):
        C[i][monomial_index[real_C[i]][j][0]][monomial_index[real_C[i]][j][1]] = 1
for i in range(len(real_C)):
    C[i]=C[i]+C[i].T
C_ = np.zeros([len(complex_C),len(monomial_set),len(monomial_set)], dtype=int)
for i in range(len(complex_C)):
    for j in range(len(monomial_index[complex_C[i]])):
        C_[i][monomial_index[complex_C[i]][j][0]][monomial_index[complex_C[i]][j][1]] = 1
for i in range(len(complex_C)):
    C_[i]=C_[i]+C_[i].T 
M = np.zeros([len(hermitian_M),len(monomial_set),len(monomial_set)], dtype=int)
for i in range(len(hermitian_M)):
    for j in range(len(monomial_index[hermitian_M[i]])):
        M[i][monomial_index[hermitian_M[i]][j][0]][monomial_index[hermitian_M[i]][j][1]] = 1
for i in range(len(hermitian_M)):
    M[i]=M[i]+M[i].T
M_ = np.zeros([len(complex_M),len(monomial_set),len(monomial_set)], dtype=int)
for i in range(len(complex_M)):
    for j in range(len(monomial_index[complex_M[i]])):
        M_[i][monomial_index[complex_M[i]][j][0]][monomial_index[complex_M[i]][j][1]] = 1
for i in range(len(complex_M)):
    M_[i]=M_[i]+M_[i].T
O = np.zeros([len(monomial_set),len(monomial_set)], dtype=int)


In [17]:
#C__=np.zeros([len(constrained_el)-1, len(monomial_set),len(monomial_set)], dtype=int)
#for i in range(1, len(constrained_el)):
#    C__[i-1]=sum(C[j] for j in range(constrained_el[i-1], constrained_el[i]))

In [18]:
diz_1 = {real_C[i] : C[i].astype(bool) for i in range(len(real_C))}
diz_2 = {complex_C[i] : C_[i].astype(bool) for i in range(len(complex_C))}
diz_3 = {hermitian_M[i] : M[i].astype(bool) for i in range(len(hermitian_M))}
diz_4 = {complex_M[i] : M_[i].astype(bool) for i in range(len(complex_M))}
diz = diz_1 | diz_2 | diz_3 | diz_4

In [19]:
complex_C_pairs=[]
for i in complex_C:
    lista = [i[j:j+n] for j in range(0, len(i), n)]
    lista.reverse()
    lista=''.join(lista)
    for tmp in cyclic_permutations(lista):
        if (tmp<lista):
            lista=tmp
    if(lista in complex_C):
        complex_C_pairs.append([complex_C.index(i), complex_C.index(lista)])
for i in range(len(complex_C_pairs)):
    complex_C_pairs[i]=sorted(complex_C_pairs[i])
complex_C_pairs=np.unique(complex_C_pairs, axis=0)

complex_M_pairs=[]
for i in complex_M:
    lista = [i[j:j+n] for j in range(0, len(i), n)]
    lista.reverse()
    lista=''.join(lista)
    flag=0
    if(lista[0:3] != 'R00'):
        for tmp in cyclic_permutations(lista):
            if (tmp[0:3] == 'R00'):
                tmp1=tmp
                tmp=lista
                flag=1
                break
    if(flag):
        lista=tmp1
    if(lista in complex_M):
        complex_M_pairs.append([complex_M.index(i), complex_M.index(lista)])
for i in range(len(complex_M_pairs)):
    complex_M_pairs[i]=sorted(complex_M_pairs[i])
complex_M_pairs=np.unique(complex_M_pairs, axis=0)
print(len(complex_M_pairs), len(complex_C_pairs))

complex_M_pairs = np.array([row for row in complex_M_pairs if row[0] != row[1]])
complex_C_pairs = np.array([row for row in complex_C_pairs if row[0] != row[1]])

8 1


In [20]:
tracial_pairs=[]
for item in matrices:
    if(len(item)>7):
        if(item[:3]+item[-3:]=='R00R00'):
            tmp_1=0
            tmp_2=0
            if(item in complex_M):
                tmp_1 = 1
                index_1 = complex_M.index(item)+1
                if(item[:-3] in complex_M):
                    tmp_2=1
                    index_2=complex_M.index(item[:-3])+1
                else:
                    index_2=hermitian_M.index(item[:-3])
            else:
                index_1 = hermitian_M.index(item)
                if(item[:-3] in complex_M):
                    tmp_2=1
                    index_2=complex_M.index(item[:-3])+1
                else:
                    index_2=hermitian_M.index(item[:-3])
            tracial_pairs.append([tmp_1, tmp_2, index_1, index_2])

In [21]:
M=M.astype(bool)
M_=M_.astype(bool)
C=C.astype(bool)
C_=C_.astype(bool)
O=O.astype(bool)
savemat("sic_d3_lvl_2.mat", {'sdp_matrices': M, 
                                      'complex_matrices': M_, 
                                      'coefs': C, 
                                      'complex_coefs': C_, 
                                      'complex_C_pairs':complex_C_pairs,
                                      'complex_M_pairs':complex_M_pairs,
                                      'tracial_pairs':tracial_pairs})

In [22]:
len(complex_M)

20

In [23]:
len(complex_C)

2

In [25]:
hermitian_M

['R00',
 'R00A00',
 'R00A01',
 'R00A10',
 'R00A11',
 'R00A00R00',
 'R00A01R00',
 'R00A10R00',
 'R00A11R00',
 'R00A00A01A00',
 'R00A00A11A00',
 'R00A00R00A00',
 'R00A00R00A01',
 'R00A00R00A10',
 'R00A00R00A11',
 'R00A01A00A01',
 'R00A01A10A01',
 'R00A01R00A00',
 'R00A01R00A01',
 'R00A01R00A10',
 'R00A01R00A11',
 'R00A10A01A10',
 'R00A10A11A10',
 'R00A10R00A00',
 'R00A10R00A01',
 'R00A10R00A10',
 'R00A10R00A11',
 'R00A11A00A11',
 'R00A11A10A11',
 'R00A11R00A00',
 'R00A11R00A01',
 'R00A11R00A10',
 'R00A11R00A11']